# Quantitative Life Science

## Specie coexistence in a neutral dynamics with enviormental noise

Michele Puppin - 1227474

# Simulations - Spacial effects

In [ ]:
# Import packages
import random
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np 
from joblib import Parallel, delayed
from numba import jit
import time
import networkx as nx
import math
import csv
import os

### Useful functions

In [ ]:
def mean_degree(G): # Compute the mean degree of a network 
    
    k = 0
    N = G.number_of_nodes()
    
    for i in range(N):
        k += G.degree[i]
        
    k_mean = k/N
    
    return k_mean 

In [ ]:
def rm_isolated(G): # Remove isoleted node in a graph 
    
    for i in list(nx.isolates(G)):

        node = random.choice(list(G.nodes()))

        while node == i:
            node = random.choice(list(G.nodes()))

        G.add_edges_from([(i, node)])

In [ ]:
@jit(nopython=True) # Ddecorator for optimization by Numba’s JIT compiler
def neighbours(adj,i): # Compute the list of neigbours of node i given the adjacency matrix
    
    neigh = []
    
    for j in range(adj.shape[0]):
        if adj[i,j] == 1:
            neigh.append(j)
    
    return neigh

## Relative fitness case

In [ ]:
@jit(nopython=True) # Ddecorator for optimization by Numba’s JIT compiler
def voter_model(state, adj, N, sAB, sigma, epsilon): # Update the state of the system
    
    sA = sAB + sigma * epsilon / 2.0 # Compute the number of seeds 
    sB = sAB - sigma * epsilon / 2.0
    
    removed = np.random.randint(N) # Choose uniformly the node to be replaced
    
    l = neighbours(adj,removed) # Compute the list of neighbours 
    
    nA = 0.0
    nB = 0.0
    
    for i in l:
        if state[i] == 0:
            nA += 1 # Compute how many neighbours belong to spacies A
        if state[i] == 1:
            nB += 1 # Compute how many neighbours belong to spacies B
        
    lambdaA = sA / (sA * nA + sB * nB) # Compute the fitness
    #lambdaB = sB / (sA * nA + sB * nB)
            
    pA = lambdaA * nA # Compute the probablity to be replaced by an individual of species A
    #pB = lambdaB * nB
    
    if random.random() < pA:
        state[removed] = 0 # Replace with an individual of species A with probability pA
    else: 
        state[removed] = 1 # Replace with an individual of species B with probability pB
        
    return state

In [ ]:
@jit(nopython=True) # Ddecorator for optimization by Numba’s JIT compiler
def propagator_rfc(adj, sAB, k, sigma): # Voter model dynamics

    N = adj.shape[0] # Number of individuals 
    # Initial number of individuals of species A and B
    state = np.array([0 for i in range(int(N/2))] + [1 for i in range(int(N/2))]) 
    np.random.shuffle(state) # Individuals are distributed randomly on the network  
    
    x = random.random() # Initialize state of the environment with uniform probability
    if x < 0.5:
        epsilon = -1
    else: 
        epsilon = 1
        
    t = 0.0
    dt = 1.0 / N # Time step
        
    while np.sum(state) != N and np.sum(state) != 0: # Run until species A monodominates or gets extinct
        
        state = voter_model(state, adj, N, sAB, sigma, epsilon) # Update the state
                
        t += dt
        
        if random.random() < (1.0 - np.exp(-k*dt)): # Cheange the environment with probability given by rate k
            epsilon = epsilon*(-1.0)
    
    return t

## 1) Lattice

In [ ]:
def ext_mean_time_latt_rfc(N, tau, niterations): # Run the dynamics on a 2D square lattice

    sAB = 0.5 # Initialize the number of seed
    k = 1.0 / ( 2.0 * tau ) # Compute the rate of the state of the environmental 
    sigma = 0.25 # Initialize the variability of the environment
    
    filename = "latt_rfc_tau"+str(tau)+".txt"

    if os.path.exists(filename): 
        os.remove(filename)

    for i in range(len(N)): # Run Voter model dynamics for each population size
        print("N: ", N[i])
        
        G_latt = nx.grid_2d_graph(N[i], N[i], periodic=False) # Create a 2D square lattice
    
        G_latt.add_edges_from([ # Add edges to connect second neighbours
            ((x, y), (x+1, y+1))
            for x in range(N[i]-1)
            for y in range(N[i]-1)
        ] + [
            ((x+1, y), (x, y+1))
            for x in range(N[i]-1)
            for y in range(N[i]-1)
        ])
        adj = nx.to_numpy_matrix(G_latt) # Compute adjacency matrix
        
        # Run Voter model dynamics in parallel for niterations realizations
        res = Parallel(n_jobs=-1, verbose=11)(delayed(propagator_rfc)(adj, sAB, k, sigma) for it_er in range(niterations))
                               
        with open(filename, "a") as f: # Store mean values of extintion time between realizations
            writer = csv.writer(f)
            writer.writerow((N[i]**2, np.mean(res)))

In [ ]:
N = [10, 14, 22, 32, 46, 68, 100, 146, 216, 316] # List of population size
tau_list = [1.0/16.0, 1.0/8.0, 1.0/4.0, 1.0/2.0, 1.0, 2.0, 4.0] # List of correlation time values
a = [3, 4, 5, 6, 7, 7, 7]
niterations = 100 # Number of realizations

for i in range(len(tau_list)):
    print("Tau: ", tau_list[i])
    ext_mean_time_latt_rfc(N[0:a[i]], tau_list[i], niterations)

## 2) Erdos-Renji

In [ ]:
def ext_mean_time_er_rfc(N, tau, niterations): # Run the dynamics on Erdos-Renj graph

    sAB = 0.5 # Initialize the number of seed
    k = 1.0 / ( 2.0 * tau ) # Compute the rate of the state of the environmental 
    sigma = 0.25 # Initialize the variability of the environment
    
    filename = "er_rfc_tau"+str(tau)+".txt"

    if os.path.exists(filename): 
        os.remove(filename)

    for i in range(len(N)): # Run Voter model dynamics for each population size
        print("N: ", N[i])
        
        deg = 8
        p = deg/(N[i]-1)
        G_ER = nx.erdos_renyi_graph(N[i],p) # Create a Erdos-Renj graph
        rm_isolated(G_ER)
        print(mean_degree(G_ER)) # Check the degree
        adj = nx.to_numpy_matrix(G_ER) # Compute adjacency matrix
        
        # Run Voter model dynamics in parallel for niterations realizations
        res = Parallel(n_jobs=-1, verbose=11)(delayed(propagator_rfc)(adj, sAB, k, sigma) for it_er in range(niterations))

        with open(filename, "a") as f: # Store mean values of extintion time between realizations
            writer = csv.writer(f)
            writer.writerow((N[i], np.mean(res)))

In [ ]:
N = [math.floor(i/2)*2 for i in np.logspace(2,5,10)] # List of population size
tau_list = [1.0/16.0, 1.0/8.0, 1.0/4.0, 1.0/2.0, 1.0, 2.0, 4.0] # List of correlation time values
a = [3, 4, 5, 6, 7, 7, 7]
niterations = 100 # Number of realizations

for i in range(len(tau_list)): # Run for different values of correlation time
    print("Tau: ", tau_list[i])
    ext_mean_time_er_rfc(N[0:a[i]], tau_list[i], niterations)

## 3) Scale-free

In [ ]:
def ext_mean_time_sf_rfc(N, tau, niterations): # Run the dynamics on Scale Free graph

    sAB = 0.5 # Initialize the number of seed
    k = 1.0 / ( 2.0 * tau ) # Compute the rate of the state of the environmental 
    sigma = 0.25 # Initialize the variability of the environment
    
    filename = "sf_rfc_tau"+str(tau)+".txt"

    if os.path.exists(filename): 
        os.remove(filename)

    for i in range(len(N)): # Run Voter model dynamics for each population size
        print("N: ", N[i])
        
        G_SF = nx.barabasi_albert_graph(N[i],4) # Create a Barabasi Albert graph
        rm_isolated(G_SF)
        print(mean_degree(G_SF))  # Check the degree
        adj = nx.to_numpy_matrix(G_SF) # Compute adjacency matrix
        
        # Run Voter model dynamics in parallel for niterations realizations
        res = Parallel(n_jobs=-1, verbose=11)(delayed(propagator_rfc)(adj, sAB, k, sigma) for it_er in range(niterations))


        with open(filename, "a") as f: # Store mean values of extintion time between realizations
            writer = csv.writer(f)
            writer.writerow((N[i], np.mean(res)))

In [ ]:
N = [math.floor(i/2)*2 for i in np.logspace(2,5,10)] # List of population size
tau_list = [1.0/16.0, 1.0/8.0, 1.0/4.0, 1.0/2.0, 1.0, 2.0, 4.0] # List of correlation time values
a = [3, 4, 5, 6, 7, 7, 7]
niterations = 100 # Number of realizations

for i in range(len(tau_list)): # Run for different values of correlation time
    print("Tau: ", tau_list[i])
    ext_mean_time_sf_rfc(N[0:a[i]], tau_list[i], niterations)

## 4) Watts-Strogatz

In [ ]:
def ext_mean_time_ws_rfc(N, tau, niterations): # Run the dynamics on Watts-Strogatz graph

    sAB = 0.5 # Initialize the number of seed
    k = 1.0 / ( 2.0 * tau ) # Compute the rate of the state of the environmental 
    sigma = 0.25 # Initialize the variability of the environment
    
    filename = "ws_rfc_tau"+str(tau)+".txt"

    if os.path.exists(filename): 
        os.remove(filename)

    for i in range(len(N)): # Run Voter model dynamics for each population size
        print("N: ", N[i])
        
        G_WS = nx.watts_strogatz_graph(N[i],8,0.05) # Create a Watts-Strogatz graph
        rm_isolated(G_WS)
        print(mean_degree(G_WS))  # Check the degree
        adj = nx.to_numpy_matrix(G_WS) # Compute adjacency matrix
        
        # Run Voter model dynamics in parallel for niterations realizations
        res = Parallel(n_jobs=-1, verbose=11)(delayed(propagator_rfc)(adj, sAB, k, sigma) for it_er in range(niterations))


        with open(filename, "a") as f: # Store mean values of extintion time between realizations
            writer = csv.writer(f)
            writer.writerow((N[i], np.mean(res)))

In [ ]:
N = [math.floor(i/2)*2 for i in np.logspace(2,5,10)] # List of population size
tau_list = [1.0/16.0, 1.0/8.0, 1.0/4.0, 1.0/2.0, 1.0, 2.0, 4.0] # List of correlation time values
a = [3, 4, 5, 6, 7, 7, 7]
niterations = 100 # Number of realizations

for i in range(len(tau_list)): # Run for different values of correlation time
    print("Tau: ", tau_list[i])
    ext_mean_time_ws_rfc(N[0:a[i]], tau_list[i], niterations)